In [17]:
import pymssql

# Define your connection parameters
server_name = 'sqlserver' # try to change to sqlserver if you encouter connection error
database_name = 'AdventureworksDWDemo'
username = 'sa'
password = 'YourStrongPassw0rd'

In [18]:
conn = pymssql.connect(server_name, username, password, database_name)

cursor = conn.cursor()

#Create tables and relationships between tables
script_parts = [
    "USE AdventureworksDWDemo",
    "IF OBJECT_ID('DimCustomer', 'U') IS NOT NULL DROP TABLE DimCustomer",
    "IF OBJECT_ID('DimProduct', 'U') IS NOT NULL DROP TABLE DimProduct",
    "IF OBJECT_ID('DimStores', 'U') IS NOT NULL DROP TABLE DimStores",
    "IF OBJECT_ID('DimSalesPerson', 'U') IS NOT NULL DROP TABLE DimSalesPerson",
    "IF OBJECT_ID('FactProductSales', 'U') IS NOT NULL DROP TABLE FactProductSales",
    "CREATE TABLE DimCustomer (CustomerID int PRIMARY KEY IDENTITY, CustomerAltID varchar(50) NOT NULL, CustomerName varchar(256), Gender varchar(20))",
    "CREATE TABLE DimProduct (ProductKey int PRIMARY KEY IDENTITY, ProductAltKey varchar(10) NOT NULL, ProductName varchar(100), ProductActualCost money, ProductSalesCost money)",
    '''
      CREATE TABLE DimStores
    (
        StoreID int PRIMARY KEY IDENTITY,
        StoreAltID varchar(10) NOT NULL,
        StoreName varchar(100),
        StoreLocation varchar(100),
        City varchar(100),
        State varchar(100),
        Country varchar(100)
    )
    ''',
    '''
    CREATE TABLE DimSalesPerson
    (
        SalesPersonID int PRIMARY KEY IDENTITY,
        SalesPersonAltID varchar(10) NOT NULL,
        SalesPersonName varchar(100),
        StoreID int,
        City varchar(100),
        State varchar(100),
        Country varchar(100)
    )
    ''',
    '''
    CREATE TABLE FactProductSales
    (
        TransactionId bigint PRIMARY KEY IDENTITY,
        SalesInvoiceNumber int NOT NULL,
        StoreID int NOT NULL,
        CustomerID int NOT NULL,
        ProductID int NOT NULL,
        SalesPersonID int NOT NULL,
        Quantity float,
        SalesTotalCost money,
        ProductActualCost money,
        Deviation float
    )
    ''',
    '''
    ALTER TABLE FactProductSales ADD CONSTRAINT FK_StoreID FOREIGN KEY (StoreID) REFERENCES DimStores(StoreID)
    ''',
    '''
    ALTER TABLE FactProductSales ADD CONSTRAINT FK_CustomerID FOREIGN KEY (CustomerID) REFERENCES DimCustomer(CustomerID)
    ''',
    '''
    ALTER TABLE FactProductSales ADD CONSTRAINT FK_ProductKey FOREIGN KEY (ProductID) REFERENCES DimProduct(ProductKey)
    ''',
    '''
    ALTER TABLE FactProductSales ADD CONSTRAINT FK_SalesPersonID FOREIGN KEY (SalesPersonID) REFERENCES DimSalesPerson(SalesPersonID)
    '''
]

for part in script_parts:
    try:
        cursor.execute(part)
        conn.commit()  # Commit changes for DDL statements
    except Exception as e:
        print(f"Error executing SQL script: {e}")
        break  # Stop execution on error

# Close the connection
cursor.close()
conn.close()

In [20]:
import pandas as pd

from sqlalchemy import create_engine

connection_string = f"mssql+pymssql://{username}:{password}@{server_name}/{database_name}"
engine = create_engine(connection_string)

from pathlib import Path
# data path is current path's parent and then AdventureWorkDWDemo
# You may need to change the path to the folder you've unzipped.

# give error
# script_path = Path.cwd().parent
# data_path = script_path / "data" / "AdventureWorkDWDemo"

dim_customer_csv = Path("data/AdventureWorkDWDemo/DimCustomer.csv")


customer_df = pd.read_csv(dim_customer_csv, header=None)
customer_df

customer_df.columns = ["CustomerAltID", "CustomerName", "Gender"]

customer_df.to_sql("DimCustomer", con=engine, if_exists="append", index=False)

# confirm it via run query 
conn = pymssql.connect(server_name, username, password, database_name)
cursor = conn.cursor()

cursor.execute("SELECT * FROM DimCustomer")

# Fetch all rows
rows = cursor.fetchall()

if rows:
    # Print each row
    for row in rows:
        print(row)

(1, 'IMI-001', 'Henry Ford', 'M')
(2, 'IMI-002', 'Bill Gates', 'M')
(3, 'IMI-003', 'Muskan Shaikh', 'F')
(4, 'IMI-004', 'Richard Thrubin', 'M')
(5, 'IMI-005', 'Emma Wattson', 'F')


In [16]:
import atoti as tt

Welcome to Atoti 0.8.10!

By using this community edition, you agree with the license available at https://docs.atoti.io/latest/eula.html.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

Atoti collects telemetry data, which is used to help understand how to improve the product.
If you don't wish to send usage data, you can request a trial license at https://www.atoti.io/evaluation-license-request.

You can hide this message by setting the `ATOTI_HIDE_EULA_MESSAGE` environment variable to True.


In [35]:
session = tt.Session()

Closing existing "Unnamed" session to create the new one.


In [36]:
session.link

http://localhost:37623

_Note_: This is the session's local URL: it may not be reachable if Atoti is running on another machine.